In [1]:
import tensorflow as tf
import numpy as np

In [2]:
csv = 'MonthlyAverageAirPollutionInSeoul.csv'

seoul_map = [ # 9 by 8 matrix, 25 borough
    [0, 0, 0, 0, '도봉구', '노원구', 0, 0],
    [0, 0, 0, '강북구', '강북구', '노원구', '노원구', 0],
    [0, '은평구', '종로구', '성북구', '성북구', '성북구', '중랑구', 0],
    [0, '은평구', '서대문구', '종로구', '종로구', '동대문구', '중랑구', 0],
    [0, '은평구', '서대문구', '서대문구', '중구', '성동구', '광진구', '강동구'],
    [0, '마포구', '마포구', '마포구', '용산구', '강남구', '송파구', '강동구'],
    ['강서구', '강서구', '영등포구', '동작구', '서초구', '강남구', '송파구', 0],
    [0, '양천구', '영등포구', '관악구', '서초구', '강남구', '송파구', 0],
    [0, '구로구', '금천구', '관악구', '서초구', 0, 0, 0]
]

with open(csv) as f:
    raw_data = f.read().strip()

In [3]:
del_quote = raw_data.replace("\"", '')
data = list(map(lambda x: x.split(','), del_quote.split('\n')))[1:]

splitted = []

ptr = 0
for i in range(len(data) // 39):
    splitted.append(data[ptr:ptr+39])
    ptr += 39
    
## test case
for date_list in splitted:
    date = date_list[0][0]
    for local in date_list:
        if date != local[0]:
            raise ValueError(date + ' is not same as ' + 'local[0]')

In [4]:
def filter_borough(dic):
    return dict(filter(lambda t: '구' in t[0], dic.items()))

pms = dict(map(lambda x: (x[0][0], dict(map(lambda x: (x[1], x[6]), x))), splitted))
pms_filtered = dict(filter(lambda x: '' not in x[1].values(), pms.items()))
pms_filtered2 = dict(map(lambda x: (x[0], filter_borough(x[1])), pms_filtered.items()))

In [5]:
def dict2seoul(p):
    return list(map(lambda t: list(map(lambda x: int(p[x]) if x != 0 else 0, t)), seoul_map))

pms_mapped = dict(map(lambda p: (p[0], dict2seoul(p[1])), pms_filtered2.items()))
pms_data = list(map(lambda x: x[1], sorted(pms_mapped.items())))
pms_result = list(map(lambda x: list(map(lambda t: t[1], sorted(x[1].items()))), sorted(pms_filtered2.items())))

In [6]:
def leaky_relu(layer):
    return tf.maximum(0.2*layer, layer)

In [7]:
keep_prob = tf.placeholder(tf.float32)

X = tf.placeholder(tf.float32, [None, 9, 8, 1])
Y = tf.placeholder(tf.float32, [None, 25])

In [8]:
w1 = tf.get_variable('w1', shape=[3, 3, 1, 32],
                    initializer=tf.random_normal_initializer())
w2 = tf.get_variable('w2', shape=[3, 3, 32, 64],
                    initializer=tf.random_normal_initializer())
w3 = tf.get_variable('w3', shape=[3, 3, 64, 128],
                    initializer=tf.random_normal_initializer())

l1 = tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME')
l1 = tf.contrib.layers.batch_norm(leaky_relu(l1))

l2 = tf.nn.conv2d(l1, w2, strides=[1, 1, 1, 1], padding='SAME')
l2 = tf.contrib.layers.batch_norm(leaky_relu(l2))

l3 = tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding='SAME')
l3 = tf.contrib.layers.batch_norm(leaky_relu(l3))